In [2]:
# Imports
import util
import random
import numpy as np
import keys
import os
import random
from openai import OpenAI
import constants

In [3]:
# Load data and Initialize OpenAI
os.environ['OPENAI_API_KEY'] = keys.WX
client = OpenAI()
client.api_key = os.getenv('OPENAI_API_KEY')

dataset, datakey = util.load_data()
#TODO split data
wp_all = dataset[0]
# sp_test = dataset[1]
sp_all = dataset[2]
# wp_test = dataset[3]
wp_base, wp_sr, wp_cr = util.split_data_by_type(wp_all)
sp_base, sp_sr, sp_cr = util.split_data_by_type(sp_all)

# Load embeddings
embedset, embedkey = util.load_data(path=constants.EMBEDDINGS)

Loaded WP-train.npy at index 0
Loaded SP_eval_data_for_practice.npy at index 1
Loaded SP-train.npy at index 2
Loaded WP_eval_data_for_practice.npy at index 3
Loaded ada-sp_all_2023-12-07.npy at index 0
Loaded ada-wp_all_2023-12-07.npy at index 1


In [4]:
# Split data
embeddings = []
ids = []
labels = []
for i in embedset[0]:
    embeddings.append(i["embedding"])
    ids.append(i["id"])
    labels.append(i["label"])
data = list(zip(embeddings, ids, labels))
train, dev = util.split_data(data, 0.8)
sp_train_embeddings, sp_train_ids, sp_train_labels = zip(*train)
sp_dev_embeddings, sp_dev_ids, sp_dev_labels = zip(*dev)
sp_train_embeddings = list(sp_train_embeddings)
sp_dev_embeddings = list(sp_dev_embeddings)
sp_train_ids = list(sp_train_ids)
sp_dev_ids = list(sp_dev_ids)
sp_train_labels = list(sp_train_labels)
sp_dev_labels = list(sp_dev_labels)

In [5]:
# Try linear classifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(sp_train_embeddings, sp_train_labels)

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
predictions = clf.predict(sp_dev_embeddings)
f1 = f1_score(sp_dev_labels, predictions, average='macro')
accuracy = accuracy_score(sp_dev_labels, predictions)
print("Accuracy: ", accuracy)
print("F1: ", f1)

Accuracy:  0.4803921568627451
F1:  0.3874779507630953


In [7]:
# log results
answers = [int(i) for i in predictions]
util.log_results(answers,
                sp_dev_ids,
                sp_all,
                "sklearn-linear",
                f1,
                accuracy)

In [ ]:
# Try Naive Bayes? CRF? BiLSTM?

In [3]:
# Model settings
MODEL = "gpt-3.5-turbo"
SYSTEM_PROMPT = {"role": "system", 
                 "content": "You are a Question Answering Model, your response must be a number from the choices that are delimited by the symbol \";\" ."}
MULTI_PREFIX = "You are a Question Answering Model, your response must be a number from the choices that are delimited by the symbol \";\". Here are some examples: \n"
SP_QUESTION = "Think outside of the box and respond with the number corresponding to the best choice for the following question.\nQuestion: "
WP_QUESTION = "For the following word problem, look at the meaning and letters in the words and respond with the number corresponding to the best choice.\nQuestion: "

In [5]:
# Multi Shot
wp_train, wp_dev = util.split_data(wp_all, 0.5)
f1, accuracy = util.run_test_nofinetune(client, MODEL, wp_dev, WP_QUESTION, SYSTEM_PROMPT, MULTI_PREFIX, training_data=wp_train, m=15, n=50)
print("SP F1: ", f1)
print("SP Accuracy: ", accuracy)

SP F1:  0.4392703201970444
SP Accuracy:  0.52
